### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [2]:
os.getcwd()

'C:\\Users\\ugoyal'

### Loading and Preprocessing data

In [15]:
# Define the paths to the dataset folders
data_path = r'C:\Users\ugoyal\Documents\Image-Detection\data'
non_vehicle_folder = os.path.join(data_path, 'non-vehicles')
vehicle_folder = os.path.join(data_path, 'vehicles')

# Initialize lists to hold image data and labels
images = []
labels = []

# Counters for labeling
car_count = 0
non_car_count = 0

# Helper function to process images from a given folder
def process_images_from_folder(folder_path, is_vehicle):
    global car_count, non_car_count
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if the image couldn't be read
        
        # Resize image to 224x224 pixels
        img = cv2.resize(img, (224, 224))
        images.append(img)
        
        # Assign labels based on whether it's a vehicle or non-vehicle
        if is_vehicle:
            label = 1  # Label for car
            car_count += 1
        else:
            label = 0  # Label for non-car
            non_car_count += 1
        
        labels.append(label)

# Process images from both folders
process_images_from_folder(vehicle_folder, is_vehicle=True)
process_images_from_folder(non_vehicle_folder, is_vehicle=False)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels:", labels)


Images shape: (17760, 224, 224, 3)
Labels: [1 1 1 ... 0 0 0]


### Normalizing and Splitting data

In [16]:
# Normalize the images
images = images / 255.0

# Convert labels to categorical
labels = to_categorical(labels, num_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

### Building the Model

In [20]:
# Initialize the model
model = Sequential()

# Add the Input layer
model.add(Input(shape=(224, 224, 3)))

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Add dense layers


# Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 394272)              │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 896 (3.50 KB)

 Trainable params: 896 (3.50 KB)

 Non-trainable params: 0 (0.00 B)